In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
# load mnist dataset
import os

from ISLP import load_data
import siuba
from siuba import _, if_else, select, mutate
from ISLP.models import (ModelSpec as MS,summarize)
from ISLP import confusion_table
import pandas as pd

In [4]:
smarket = load_data('Smarket')
print(smarket.shape)

(1250, 9)


In [13]:
smarket2=(
    smarket >>
    #mutate(Direction = if_else(_.Direction == 'Up',1,0)) >>
    mutate(train = if_else(_.Year < 2005,1,0) ) >>
    select(~_.Year,~_.Today)

)
train = (smarket2.train == 1)
# print(smarket2.head())

# print(design)

X = (
    smarket2 >>
    select(~_.train,~_.Direction)
)
y = smarket2['Direction']  == 'Up'

x_train, x_test = X.loc[train], X.loc[~train]
y_train, y_test = y.loc[train], y.loc[~train]
y_test_raw =y_test

In [14]:
# compute the number of labels
num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train.shape

(998, 2)

In [15]:
x_train

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
0,0.381,-0.192,-2.624,-1.055,5.010,1.1913
1,0.959,0.381,-0.192,-2.624,-1.055,1.2965
2,1.032,0.959,0.381,-0.192,-2.624,1.4112
3,-0.623,1.032,0.959,0.381,-0.192,1.2760
4,0.614,-0.623,1.032,0.959,0.381,1.2057
...,...,...,...,...,...,...
993,0.046,0.342,0.904,0.038,-0.749,0.9561
994,-0.431,0.046,0.342,0.904,0.038,0.9220
995,0.715,-0.431,0.046,0.342,0.904,0.9830
996,-0.007,0.715,-0.431,0.046,0.342,0.9259


In [16]:
# input image dimensions
image_size = x_train.shape[1]


In [19]:
# network parameters
# image is processed as is (square grayscale)
input_shape = (image_size, 1)
batch_size = 128
kernel_size = 3
pool_size = 2
filters = 64
dropout = 0.2

In [20]:
# model is a stack of CNN-ReLU-MaxPooling
model = Sequential()
model.add(Conv2D(filters=filters,
kernel_size=kernel_size,
activation='relu',
input_shape=input_shape))
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters=filters,
kernel_size=kernel_size,
activation='relu'))
model.add(MaxPooling2D(pool_size))
model.add(Conv2D(filters=filters,
kernel_size=kernel_size,
activation='relu'))
model.add(Flatten())
# dropout added as regularizer
model.add(Dropout(dropout))
# output layer is 10-dim one-hot vector
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
plot_model(model, to_file='cnn-mnist.png', show_shapes=True)

ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 6, 1)

In [20]:
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])
# train the network
model.fit(x_train, y_train, epochs=10, batch_size=batch_size)
_, acc = model.evaluate(x_test,
y_test,
batch_size=batch_size,
verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

Epoch 1/10
469/469 [==============================] - 18s 37ms/step - loss: 0.2520 - accuracy: 0.9234
Epoch 2/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0670 - accuracy: 0.9791
Epoch 3/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0467 - accuracy: 0.9859
Epoch 4/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0380 - accuracy: 0.9878
Epoch 5/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0308 - accuracy: 0.9903
Epoch 6/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0260 - accuracy: 0.9913
Epoch 7/10
469/469 [==============================] - 17s 37ms/step - loss: 0.0247 - accuracy: 0.9925
Epoch 8/10
469/469 [==============================] - 20s 43ms/step - loss: 0.0198 - accuracy: 0.9938
Epoch 9/10
469/469 [==============================] - 19s 41ms/step - loss: 0.0173 - accuracy: 0.9945
Epoch 10/10
469/469 [==============================] - 17s 37ms/step - loss: 0.017